In [ ]:
import erum_data_data as edd
import tensorflow as tf
from utils import generate_model
from utils import train_plots
from utils import roc_auc
from utils import test_accuracy
from utils import test_f1_score

In [ ]:
datasets = [ # uncomment the dataset name you want to train the model on.                           
#            'EOSL', 
#            'spinodal', 
            'airshower', 
            'LHCO',
            'top',
#            'belle'
           ]

for ds in datasets:
    X_train, y_train  = edd.load(ds, dataset='train', cache_dir = './'+ ds, cache_subdir = 'datasets')
    X_test, y_test = edd.load(ds, dataset='test', cache_dir = './'+ ds, cache_subdir = 'datasets')
    shape1 = X_train[0].shape[1:]
    shape2 = X_train[1].shape[1:] if len(X_train) > 1 else 0
    
    model = generate_model(ds, 15, 256, shape1, shape2)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy(name = "acc"), tf.keras.metrics.AUC(name = "AUC")],)

    history = model.fit(
                        x = X_train, y = y_train,
                        shuffle=True,
                        validation_split = 0.3,
                        epochs=100,
                        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta =0.0001, patience=15, restore_best_weights = True)],
                        batch_size = 300,
                        verbose = 1
                    )
    
    #training history plots
    train_plots(history, ds)
    
    #evaluation plots and scores
    y_pred = model.predict(X_test).ravel()
    roc_auc(y_pred, y_test, ds, True)
    test_accuracy(y_pred, y_test, ds)
    test_f1_score(y_pred, y_test, ds)